In [1]:
import os
import shutil
import random
import pandas as pd

In [2]:
Clasif = pd.read_csv('Carac_Stars.csv')
Clasif

,Kepler_ID,Metalicidad,Clasificacion,incert_clas
0,10797460,0.136,CANDIDATE,1.000
1,10811496,-0.127,FALSE POSITIVE,0.000
2,10848459,-0.522,FALSE POSITIVE,0.000
3,10854555,0.081,CANDIDATE,1.000
4,10872983,-0.067,CANDIDATE,0.992
...,...,...,...,...
7677,10128825,-0.046,CANDIDATE,0.497
7678,10147276,-0.038,FALSE POSITIVE,0.021
7679,10156110,0.082,FALSE POSITIVE,0.000
7680,8460634,-0.556,FALSE POSITIVE,0.000


In [3]:
Clasif[Clasif['Kepler_ID']==10000162]['Clasificacion']

3845    CANDIDATE
Name: Clasificacion, dtype: object

In [4]:
def mover_archivos(archivos:list, vieja_carpeta:str, nueva_carpeta:str):
    """Recibe una lista de archivos, la carpeta actual y el lugar a donde se moveran"""
    for archivo in archivos:
        shutil.move(vieja_carpeta+'/'+archivo, nueva_carpeta+'/'+archivo)

def segmentacion_estrellas(base_dir:str, df_clasif, columna_clasif:str, columna_ID:str):
    """Se separan los datos en dos carpetas ubicadas en la carpeta orginal, 
    una carpeta con los candidatos a tener exoplanetas y otra a los que no 
    poseen"""
    # Listar todos los archivos en la carpeta base
    archivos = [f for f in os.listdir(base_dir) if f.endswith('.csv') and os.path.isfile(os.path.join(base_dir, f))]
    archivos_1 = []
    archivos_0 = []
    for archivo in archivos:
        print(archivo)
        print(archivo[3:-4])
        if df_clasif[df_clasif[columna_ID]==int(archivo[3:-4])][columna_clasif].iloc[0]=='CANDIDATE':
            archivos_1.append(archivo)
        else:
            archivos_0.append(archivo)
    mover_archivos(archivos_1, base_dir, base_dir+'/1')
    mover_archivos(archivos_0, base_dir, base_dir+'/0')
    
def modeling_div(base_dir:str, proporciones, new_carpeta:str):
    # Listar todos los archivos en la carpeta base
    archivos = [f for f in os.listdir(base_dir)]
    seed = random.Random(1312)
    seed.shuffle(archivos)  # Mezclar los archivos aleatoriamente
    
    # Calcular cantidades de archivos para cada subcarpeta
    total_archivos = len(archivos)
    num_train = int(total_archivos * proporciones['train'])
    num_val = int(total_archivos * proporciones['val'])
    
    # Dividir los archivos en las subcarpetas correspondientes
    archivos_train = archivos[:num_train]
    archivos_val = archivos[num_train:num_train + num_val]
    archivos_test = archivos[num_train + num_val:]
    
    mover_archivos(archivos_train, base_dir, new_carpeta['train'])
    mover_archivos(archivos_val, base_dir, new_carpeta['val'])
    mover_archivos(archivos_test, base_dir, new_carpeta['test'])

In [9]:
old_dir = '../Curvas_de_luz_God'

proporciones = {'train': 0.7, 
                'val': 0.2, 
                'test': 0.1}

new_carpeta_0 = {'train': '../Datos_Exoplanetas/0/Train', 
               'val':'../Datos_Exoplanetas/0/Validation', 
               'test':'../Datos_Exoplanetas/0/Test'}

new_carpeta_1 = {'train': '../Datos_Exoplanetas/1/Train', 
               'val':'../Datos_Exoplanetas/1/Validation', 
               'test':'../Datos_Exoplanetas/1/Test'}

In [6]:
segmentacion_estrellas(old_dir, Clasif, 'Clasificacion', 'Kepler_ID')

LC_9777251.csv
9777251
LC_8415745.csv
8415745
LC_9528420.csv
9528420
LC_5009189.csv
5009189
LC_9161118.csv
9161118
LC_9099950.csv
9099950
LC_10861608.csv
10861608
LC_9008220.csv
9008220
LC_3644523.csv
3644523
LC_6150977.csv
6150977
LC_11026582.csv
11026582
LC_5872150.csv
5872150
LC_3438507.csv
3438507
LC_11087662.csv
11087662
LC_9642018.csv
9642018
LC_6620003.csv
6620003
LC_8649876.csv
8649876
LC_10214873.csv
10214873
LC_8261920.csv
8261920
LC_12406807.csv
12406807
LC_11565924.csv
11565924
LC_9264949.csv
9264949
LC_10266860.csv
10266860
LC_5906426.csv
5906426
LC_5702770.csv
5702770
LC_3446451.csv
3446451
LC_7362696.csv
7362696
LC_3458919.csv
3458919
LC_9579860.csv
9579860
LC_11513486.csv
11513486
LC_11805075.csv
11805075
LC_7125794.csv
7125794
LC_5460835.csv
5460835
LC_8866137.csv
8866137
LC_4548098.csv
4548098
LC_10331279.csv
10331279
LC_9940565.csv
9940565
LC_4857213.csv
4857213
LC_5938970.csv
5938970
LC_10679583.csv
10679583
LC_10156110.csv
10156110
LC_5025217.csv
5025217
LC_8912730

In [10]:
modeling_div(old_dir+'/1', proporciones, new_carpeta_1)

In [11]:
modeling_div(old_dir+'/0', proporciones, new_carpeta_0)